In [6]:
import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow


# Input video path
input_video = "/content/drive/MyDrive/Colab Notebooks/Computer Vision/WebCam&Video_blur_&_Text_detection/face_detect_video.mp4"
# Output directory to save the processed video
output_dir = "/content/drive/MyDrive/Colab Notebooks/Computer Vision/WebCam&Video_blur_&_Text_detection/output"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
output_video = os.path.join(output_dir, "blured_face_video.mp4")

# Check if the input video file exists
if not os.path.exists(input_video):
    print("Error: Video not found")
    exit(1)

# Initialize Mediapipe face detection model
mp_face = mp.solutions.face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Open the video file for reading
cap = cv2.VideoCapture(input_video)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
    exit(1)

# Get the width, height, and frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 0.50)  # Resize frame width to 50% of original
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 0.50)  # Resize frame height to 50% of original
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get the frames per second of the video


fourcc = cv2.VideoWriter_fourcc(*"MP4V")
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to reduce processing time
    frame = cv2.resize(frame, (frame_width, frame_height))

    # Convert the frame from BGR to RGB for face detection
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mp_face.process(frame_rgb)  # Detect faces in the frame

    # If faces are detected in the frame
    if results.detections:
        for detection in results.detections:
            # Get the bounding box coordinates for the detected face
            bbox = detection.location_data.relative_bounding_box
            x, y, w, h = (
                int(bbox.xmin * frame_width), int(bbox.ymin * frame_height),
                int(bbox.width * frame_width), int(bbox.height * frame_height)
            )

            # Add a little padding around the detected face
            x, y = max(0, x - 10), max(0, y - 10)
            w, h = min(frame_width - x, w + 20), min(frame_height - y, h + 20)


            face_roi = frame[y:y+h, x:x+w]

            # Apply Gaussian blur to the face region
            if face_roi.shape[0] > 0 and face_roi.shape[1] > 0:
                blured_face = cv2.GaussianBlur(face_roi, (55, 55), 30)
                frame[y:y+h, x:x+w] = blured_face

    # Write the modified frame to the output video file
    out.write(frame)

    # Display the current fram
    cv2_imshow(frame) #(for google colab:cv2_imshow)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture and output video objects
cap.release()
out.release()

# Close any OpenCV windows
cv2.destroyAllWindows()

# Print a message
print(f"Blurred face video saved at {output_video}")


Output hidden; open in https://colab.research.google.com to view.